# **Stock Kalman Filtering Techniques**

In [7]:
import numpy as np
import pandas as pd
from math import sqrt
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [8]:
start = datetime(2017, 1, 1)
end = datetime.today()

stock = yf.download('TSLA', start = start, end = end)
stock.reset_index(inplace = True)

[*********************100%%**********************]  1 of 1 completed


In [9]:
stock

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-01-03,14.324000,14.688667,14.064000,14.466000,14.466000,88849500
1,2017-01-04,14.316667,15.200000,14.287333,15.132667,15.132667,168202500
2,2017-01-05,15.094667,15.165333,14.796667,15.116667,15.116667,88675500
3,2017-01-06,15.128667,15.354000,15.030000,15.267333,15.267333,82918500
4,2017-01-09,15.264667,15.461333,15.200000,15.418667,15.418667,59692500
...,...,...,...,...,...,...,...
1697,2023-10-02,244.809998,254.279999,242.619995,251.600006,251.600006,123810400
1698,2023-10-03,248.610001,250.020004,244.449997,246.529999,246.529999,101985300
1699,2023-10-04,248.139999,261.859985,247.600006,261.160004,261.160004,129721600
1700,2023-10-05,260.000000,263.600006,256.250000,260.049988,260.049988,119159200


In [10]:
# 1. Kalman_Filter Class

def Kalman_Filter(Y):
    S = Y.shape[0]
    S = S + 1
    "Initialize Params:"
    Z = param0[0]
    T = param0[1]
    H = param0[2]
    Q = param0[3]
    # "Kalman Filter Starts:"
    u_predict = np.zeros(S)
    u_update = np.zeros(S)
    P_predict = np.zeros(S)
    P_update = np.zeros(S)
    v = np.zeros(S)
    F = np.zeros(S)
    KF_Dens = np.zeros(S)
    for s in range(1,S):
        if s == 1:
            P_update[s] = 1000
            P_predict[s] =  T*P_update[1]*np.transpose(T)+Q
        else:
            F[s]= Z*P_predict[s-1]*np.transpose(Z)+H
            v[s] = Y[s-1] - Z*u_predict[s-1]
            u_update[s] = u_predict[s-1]+P_predict[s-1]*np.transpose(Z)*(1/F[s])*v[s]
            u_predict[s] = T*u_predict[s];
            P_update[s] = P_predict[s-1]-P_predict[s-1]*np.transpose(Z)*(1/F[s])*Z*P_predict[s-1]
            P_predict[s] = T*P_update[s]*np.transpose(T)+Q
            Likelihood = np.sum(KF_Dens[1:-1])
            return Likelihood

def Kalman_Smoother(params, Y):
    S = Y.shape[0]
    S = S + 1
    "Initialize Params:"
    Z = params[0]
    T = params[1]
    H = params[2]
    Q = params[3]
    "Kalman Filter Starts:"
    u_predict = np.zeros(S)
    u_update = np.zeros(S)
    P_predict = np.zeros(S)
    P_update = np.zeros(S)
    v = np.zeros(S)
    F = np.zeros(S)
    for s in range(1,S):
        if s == 1:
            P_update[s] = 1000
            P_predict[s] =  T*P_update[1]*np.transpose(T)+Q
        else:
            # "Please fill this part."
            F[s]= Z*P_predict[s-1]*np.transpose(Z)+H
            v[s] = Y[s-1] - Z*u_predict[s-1]
            u_update[s] = u_predict[s-1]+P_predict[s-1]*np.transpose(Z)*(1/F[s])*v[s]
            u_predict[s] = T*u_predict[s];
            P_update[s] = P_predict[s-1]-P_predict[s-1]*np.transpose(Z)*(1/F[s])*Z*P_predict[s-1]
            P_predict[s] = T*P_update[s]*np.transpose(T)+Q


            u_smooth = np.zeros(S)
            P_smooth = np.zeros(S)
            u_smooth[S-1] = u_update[S-1]
            P_smooth[S-1] = P_update[S-1]
    for  t in range(S-1,0,-1):

        u_smooth[t-1] = u_update[t] +P_update[t]*np.transpose(T)/P_predict[t]*(u_smooth[t]- T*u_update[t])
        P_smooth[t-1] = P_update[t] + (P_update[t]*np.transpose(T)/P_predict[t]*(P_smooth[t]-P_update[t])/P_update[t]*T*P_update[t])

    u_smooth = u_smooth[0:-1]
    return u_smooth

In [11]:
Y = stock['Adj Close'].values
T = Y.size

param0 = np.array([0.78, 1.25, 160*np.std(Y), 920*np.std(Y)])
param_star = minimize(Kalman_Filter, param0, method = 'BFGS', options = {'xtol': 1e-8, 'disp': True})
y_pred = Kalman_Smoother(param_star.x, Y)
timevec = np.linspace(1, T, T)

'''fig = plt.figure(figsize = (20, 8))
plt.plot(timevec, Y, 'r-', label = 'Actual')
plt.plot(timevec, y_pred, 'b:', label = 'Predicted')
plt.legend(loc = 'upper right')
plt.title("Kalman Filtering")
plt.show()'''
import plotly.graph_objects as go
fig = go.Figure()

# Add actual trace
fig.add_trace(go.Scatter(
    x=timevec,
    y=Y,
    mode='lines',
    name='Actual'
)) 

# Add predicted trace  
fig.add_trace(go.Scatter(
    x=timevec,
    y=y_pred,  
    mode='lines',
    name='Predicted'
))

# Set title
fig.update_layout(
    title='Kalman Filtering',
    xaxis_title='Timevec',
    yaxis_title='Price'
)

# Show figure
fig.show()

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 5
         Gradient evaluations: 1


C:\Users\aksha\AppData\Local\Temp\ipykernel_27988\871030641.py:5: OptimizeWarning: Unknown solver options: xtol
  param_star = minimize(Kalman_Filter, param0, method = 'BFGS', options = {'xtol': 1e-8, 'disp': True})


In [12]:
results = pd.DataFrame({'Actual': list(Y), 'Predicted': list(y_pred), 'Date': stock['Date'], 'Open': stock['Open'], 'Close': stock['Close']})
results.set_index('Date', inplace = True)
results.head()

,Actual,Predicted,Open,Close
Date,,,,
2017-01-03,14.466000,0.168218,14.324000,14.466000
2017-01-04,15.132667,14.313917,14.316667,15.132667
2017-01-05,15.116667,14.938807,15.094667,15.116667
2017-01-06,15.267333,15.105781,15.128667,15.267333
2017-01-09,15.418667,15.209075,15.264667,15.418667



**Long Short Day trading:**

*   if predicted > yesterdays close, buy and sell at end of day
*   if predicted < yesterdays close, sell and buy at end of day



In [13]:
amount = 10000
signal = 0
Amount = []
balance = 0
action = []
portfolio = 0
Portfolio = []
stocks = 0
Stocks = []

for i in range(len(results) - 1):
    if results['Predicted'][i] > results['Actual'][i-1]:
        action.append('Buy at Open & Sell at Close')
        stocks = int(amount/results['Open'][i])
        balance = int(amount%results['Close'][i])
        portfolio = stocks * results ['Open'][i]
        print(i, 'Buy at Open', round(portfolio, 2), stocks, round(balance, 2))

        # action.append('Sell at End')
        portfolio = stocks * results['Close'][i]
        signal = 0
        stocks = 0
        amount = balance + portfolio
        portfolio = 0
        balance = 0
        print(i, 'Sell at Close', round(amount, 2), balance)
        Amount.append(amount)
        Portfolio.append(round(portfolio, 5))

    else:
        action.append('Sell at Open & Bull at Close')
        stocks = int(amount/results['Open'][i])
        balance = int(amount%results['Close'][i])
        portfolio = stocks * results ['Open'][i]
        print(i, 'Sell at Open', round(portfolio, 2), '-', stocks, round(balance, 2))

        # action.append('Buy at Close')
        portfolio = stocks * results['Close'][i]
        signal = 0
        stocks = 0
        amount = balance + portfolio
        portfolio = 0
        balance = 0
        print(i, 'Buy Back at Close', round(amount, 2), balance)
        Amount.append(amount)
        Portfolio.append(round(portfolio, 5))
    print('\n')

Strategy_Result = pd.DataFrame({'Actual_Close_Price':stock.Close[1:],
                                 'Predicted_Close_Price':list(y_pred[1:]),
                                 'Date': stock.Date[1:],
                                 'Action':  action,
                                 # 'Stocks': Stocks,
                                 'Portfolio($)': Portfolio,
                                 'Amount($)':Amount
                                })

Strategy_Result.tail()
'''Strategy_Result[['Actual_Close_Price', 'Predicted_Close_Price']].loc[0:].plot(figsize = (15, 4), style = ['-', '--'], title = 'Actual vs Predicted')
Strategy_Result[['Amount($)']].loc[0:].plot(figsize = (15, 4), style = ['-g'], title = 'Total Amount')'''

import plotly.graph_objects as go

# Actual and predicted data
actual = Strategy_Result['Actual_Close_Price'] 
predicted = Strategy_Result['Predicted_Close_Price']
amount= Strategy_Result['Amount($)']
date=Strategy_Result['Date']

# Create figure
fig = go.Figure()

# Add actual trace
fig.add_trace(go.Scatter(
    x=date,
    y=actual,
    mode='lines',
    name='Actual'
)) 

# Add predicted trace  
fig.add_trace(go.Scatter(
    x=date,
    y=predicted,  
    mode='lines',
    name='Predicted'
))

# Set title
fig.update_layout(
    title='Actual vs Predicted',
    xaxis_title='Date',
    yaxis_title='Price'
)

# Show figure
fig.show()

figt=go.Figure()

figt.add_trace(go.Scatter(
    x=date,
    y=amount,
    mode='lines',
    name='Actual'
))

# Set title
figt.update_layout(
    title='Total Amount',
    xaxis_title='Date',
    yaxis_title='Price'
)

figt.show()

0 Sell at Open 9998.15 - 698 3
0 Buy Back at Close 10100.27 0


1 Sell at Open 10093.25 - 705 6
1 Buy Back at Close 10674.53 0


2 Sell at Open 10671.93 - 707 2
2 Buy Back at Close 10689.48 0


3 Sell at Open 10680.84 - 706 2
3 Buy Back at Close 10780.74 0


4 Sell at Open 10776.85 - 706 3
4 Buy Back at Close 10888.58 0


5 Sell at Open 10888.53 - 704 8
5 Buy Back at Close 10796.57 0


6 Sell at Open 10781.56 - 706 14
6 Buy Back at Close 10826.63 0


7 Sell at Open 10811.63 - 708 5
7 Buy Back at Close 10841.65 0


8 Buy at Open 10840.67 707 0
8 Sell at Close 11205.95 0


9 Sell at Open 11203.8 - 710 8
9 Buy Back at Close 11158.79 0


10 Buy at Open 11154.1 707 3
10 Sell at Close 11237.7 0


11 Buy at Open 11225.15 681 8
11 Sell at Close 11074.7 0


12 Sell at Open 11062.06 - 676 12
12 Buy Back at Close 11041.16 0


13 Buy at Open 11030.47 673 5
13 Sell at Close 11173.21 0


14 Buy at Open 11166.67 670 4
14 Sell at Close 11376.58 0


15 Sell at Open 11373.1 - 663 10
15 Buy Back at Close

In [14]:
Profit = (Amount[-1] / 10000) - 1  # Assuming Amount[-1] represents the final amount
Profit_percent = Profit * 100

print('Initial Investment: $10,000')
print('Final Amount:', Amount[-1], '$')
print('Profit Percentage:', Profit_percent, '%')

Initial Investment: $10,000
Final Amount: 20629.94903564453 $
Profit Percentage: 106.29949035644532 %
